In [1]:
from ndmg.stats import qa_graphs as qg

/home/student/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from __future__ import print_function

from collections import OrderedDict

import networkx as nx
import os


def loadGraphs(filenames, verb=False):
    """
    Given a list of files, returns a dictionary of graphs
    Required parameters:
        filenames:
            - List of filenames for graphs
    Optional parameters:
        verb:
            - Toggles verbose output statements
    """
    #  Initializes empty dictionary
    if type(filenames) is not list:
        filenames = [filenames]
    gstruct = OrderedDict()
    for idx, files in enumerate(filenames):
        if verb:
            print("Loading: " + files)
        #  Adds graphs to dictionary with key being filename
        fname = os.path.basename(files)
        try:
            gstruct[fname] = nx.read_graphml(files)
        except:
            gstruct[fname] = nx.read_gpickle(files)
    return gstruct

In [3]:
import numpy as np
from scipy.stats import gaussian_kde
from itertools import product
from plotly.graph_objs import *
from plotly import tools


def plot_heatmap(dats, name=None, ylab=None, xlab=None, scale=False,
                 scaletit=''):
    data = [
            Heatmap(
                    z=dats,
                    name=name,
                    showscale=scale,
                    colorscale='Reds',
                    colorbar=dict(title=scaletit)
                   )
           ]
    layout = std_layout(name, ylab, xlab)
    fig = Figure(data=data, layout=layout)
    return fig


def plot_degrees(dats, name=None, ylab=None, xlab=None, hemi=True):
    data = list()
    if hemi:
        main = dats['ipso_deg']
        contra = dats['contra_deg']
    else:
        main = dats['total_deg']
    al = (4.0/len(main.keys()))

    for key in main.keys():
        lgth = len(main[key])
        data += [
                 Scatter(
                         x=np.linspace(1, lgth, lgth),
                         y=main[key],
                         line=Line(
                                   color='rgba(0,0,0,%1.2f)' % al
                                  ),
                         hoverinfo='x',
                         name=name,
                        )
                ]
        if hemi:
            data += [
                     Scatter(
                             x=np.linspace(1, lgth, lgth),
                             y=contra[key],
                             line=Line(
                                       color='rgba(0.11,0.62,0.47,%1.2f)' % al
                                      ),
                             hoverinfo='x',
                             name=name,
                            )
                    ]
    layout = std_layout(name, ylab, xlab)
    fig = Figure(data=data, layout=layout)
    return fig


def plot_series(dats, name=None, ylab=None, xlab=None, sort=False):
    data = list()
    for idx, ys in enumerate(dats):
        if sort:
            ys = np.sort(ys)
        data += [
                 Scatter(
                         x=np.linspace(1, len(ys), len(ys)),
                         y=ys,
                         line=Line(
                                   color='rgba(0,0,0,%1.2f)' % (4.0/len(dats))
                                  ),
                         hoverinfo='x',
                         name=name,
                        )
                ]
    layout = std_layout(name, ylab, xlab)
    fig = Figure(data=data, layout=layout)
    return fig


def plot_density(xs, ys, name=None, ylab=None, xlab=None):
    data = list()
    for idx, x in enumerate(xs):
        data += [
                 Scatter(
                         x=xs[idx],
                         y=ys[idx],
                         line=Line(
                                   color='rgba(0,0,0,%1.2f)' % (4.0/len(ys))
                                  ),
                         hoverinfo='x',
                         name=name,
                        )
                ]
    layout = std_layout(name, ylab, xlab)
    fig = Figure(data=data, layout=layout)
    return fig


def plot_rugdensity(series, name=None, ylab=None, xlab=None):
    if len(series) > 1:
        dens = gaussian_kde(series)
        x = np.linspace(np.min(series), np.max(series), 100)
        y = dens.evaluate(x)*np.max(series)

        d_rug = Scatter(
                    x=series,
                    y=[0]*len(series),
                    mode='markers',
                    marker=Marker(
                             color='rgba(0,0,0,0.9)',
                             symbol='line-ns-open',
                             size=10,
                             opacity=0.5
                           ),
                    name=name
              )
    else:
        x = 0
        y = series

    d_dens = Scatter(
                x=x,
                y=y,
                line=Line(
                       color='rgba(0,0,0,0.9)'
                     ),
                hoverinfo='x',
                name=name,
           )
    if len(series) > 1:
        data = [d_dens, d_rug]
    else:
        data = [d_dens]
    layout = std_layout(name, ylab, xlab)
    fig = Figure(data=data, layout=layout)
    return fig


def std_layout(name=None, ylab=None, xlab=None):
    return Layout(
            title=name,
            showlegend=False,
            xaxis={'nticks': 5,
                   'title': xlab},
            yaxis={'nticks': 3,
                   'title': ylab}
          )


def fig_to_trace(fig):
    data = fig['data']
    for item in data:
        item.pop('xaxis', None)
        item.pop('yaxis', None)
    return data


def traces_to_panels(traces, names=[], ylabs=None, xlabs=None):
    r, c, locs = panel_arrangement(len(traces))
    multi = tools.make_subplots(rows=r, cols=c, subplot_titles=names)
    for idx, loc in enumerate(locs):
        if idx < len(traces):
            for component in traces[idx]:
                multi.append_trace(component, *loc)
        else:
            multi = panel_invisible(multi, idx+1)
    multi.layout['showlegend'] = False
    return multi


def panel_arrangement(num):
    dims = list()
    count = 0
    while len(dims) == 0:
        dims = list(factors(num+count))
        count += 1

    if len(dims) == 1:
        row = col = dims[0]
    else:
        row = dims[0]
        col = dims[-1]

    locations = [(a+1, b+1) for a, b in product(range(row), range(col))]
    return row, col, locations


def panel_invisible(plot, idx):
    for c in ['x', 'y']:
        axe = c+'axis'+str(idx)
        plot.layout[axe]['showgrid'] = False
        plot.layout[axe]['zeroline'] = False
        plot.layout[axe]['showline'] = False
        plot.layout[axe]['showticklabels'] = False
    return plot


def rand_jitter(arr):
    stdev = .03*(max(arr)-min(arr)+2)
    return arr + np.random.randn(len(arr)) * stdev


def factors(N):
    return set([item for subitem in
                [(i, N//i) for i in range(1, int(N**0.5) + 1)
                 if N % i == 0 and i > 1] for item in subitem])

In [4]:
fs = ['graph.graphml','graph_1.gpickle']
outdir = "/home/student/Desktop/research/output"

In [5]:
gr = loadGraphs(fs, verb=False)
gr

OrderedDict([('graph.graphml',
              <networkx.classes.graph.Graph at 0x7f3b351783d0>),
             ('graph_1.gpickle',
              <networkx.classes.graph.Graph at 0x7f3b35178450>)])

In [6]:
def binGraphs(graphs, thr=0.1):
    """
    Binarizes a set of graphs given a threshold.
    Required Parameters:
        graphs:
            - a list of graphs.
        thr:
            - the threshold below which edges will be assumed disconnected.
              .1 is chosen as default according to discriminability results.
    """
    binGraphs = {}
    for subj, graph in graphs.iteritems():
        bin_graph = nx.Graph()
        for (u, v, d) in graph.edges(data=True):
            if d['weight'] > thr:
                bin_graph.add_edge(u, v, weight=1)
        binGraphs[subj] = bin_graph
    return binGraphs

In [7]:
def density(data, nbins=500, rng=None):
    """
    Computes density for metrics which return vectors
    Required parameters:
        data:
            - Dictionary of the vectors of data
    """
    density = OrderedDict()
    xs = OrderedDict()
    for subj in data:
        hist = np.histogram(data[subj], nbins)
        hist = np.max(hist[0])
        dens = gaussian_kde(data[subj])
        if rng is not None:
            xs[subj] = np.linspace(rng[0], rng[1], nbins)
        else:
            xs[subj] = np.linspace(0, np.max(data[subj]), nbins)
        density[subj] = dens.evaluate(xs[subj])*np.max(data[subj]*hist)
    return {"xs": xs, "pdfs": density}

In [8]:
modality='dwi'
if modality == 'func':
    graphs = binGraphs(gr)
else:
    graphs = gr

In [9]:
nodes = nx.number_of_nodes(graphs.values()[0])

 #  Number of non-zero edges (i.e. binary edge count)

In [10]:
nnz = OrderedDict((subj, len(nx.edges(graphs[subj]))) for subj in graphs)
# write(outdir, 'number_non_zeros', nnz, atlas)

In [11]:
# Instead of creating a seperate pickled file, just store it in a dictionary.
statsDict = {'number_non_zeros': nnz}

In [12]:
import numpy as np
print("Computing: NNZ")
print("Sample Mean: %.2f" % np.mean(nnz.values()))

Computing: NNZ
Sample Mean: 1337.00


# Scan Statistic-1

In [13]:
def scan_statistic(mygs, i):
    """
    Computes scan statistic-i on a set of graphs
    Required Parameters:
        mygs:
            - Dictionary of graphs
        i:
            - which scan statistic to compute
    """
    ss = OrderedDict()
    for key in mygs.keys():
        g = mygs[key]
        tmp = np.array(())
        for n in g.nodes():
            sg = nx.ego_graph(g, n, radius=i)
            tmp = np.append(tmp, np.sum([sg.get_edge_data(e[0], e[1])['weight']
                            for e in sg.edges()]))
        ss[key] = tmp
    return ss

In [14]:
def show_means(data):
    print("Subject Means: " + ", ".join(["%.2f" % np.mean(data[key]) 
                                         for key in data.keys()]))

In [15]:
temp_ss1 = scan_statistic(graphs, 1)
ss1 = temp_ss1
# write(outdir, 'locality_statistic', ss1, atlas)
statsDict['locality_statistic'] = ss1
print("Computing: Max Local Statistic Sequence")
show_means(temp_ss1)

Computing: Max Local Statistic Sequence
Subject Means: 398574.53, 249819.52


In [16]:
def rankGraphs(graphs):
    """
    Ranks the edges in each element of a list of graphs.
    Required Parameters:
        graphs:
            - a list of graphs.
    """
    rankGraphs = {}
    for subj, graph in graphs.iteritems():
        rgraph = nx.Graph()
        edge_ar = np.asarray([x[2]['weight'] for x in graph.edges(data=True)])
        rank_edge = rankdata(edge_ar)  # rank the edges
        for ((u, v, d), rank) in zip(graph.edges(data=True), rank_edge):
            rgraph.add_edge(u, v, weight=rank)
        rankGraphs[subj] = rgraph
    return rankGraphs

In [17]:
if modality == 'func':
    graphs = rankGraphs(gr)
    wt_args = {'weight': 'weight'}
else:
    wt_args = {}

#   Clustering Coefficients

In [18]:
temp_cc = OrderedDict((subj, nx.clustering(graphs[subj],
                                           **wt_args).values()) 
                      for subj in graphs)
ccoefs = temp_cc
# write(outdir, 'clustering_coefficients', ccoefs, atlas)
statsDict['clustering_coefficients'] = ccoefs
print("Computing: Clustering Coefficient Sequence")
show_means(temp_cc)

Computing: Clustering Coefficient Sequence
Subject Means: 0.65, 0.65


#  Degree sequence

In [19]:
total_deg = OrderedDict((subj, np.array(nx.degree(graphs[subj],
                                                  **wt_args).values()))
                        for subj in graphs)
ipso_deg = OrderedDict()
contra_deg = OrderedDict()
for subj in graphs:  # TODO GK: remove forloop and use comprehension maybe?
    g = graphs[subj]
    N = len(g.nodes())
    LLnodes = g.nodes()[0:N/2]  # TODO GK: don't assume hemispheres
    LL = g.subgraph(LLnodes)
    LLdegs = [LL.degree(**wt_args)[n] for n in LLnodes]

    RRnodes = g.nodes()[N/2:N]  # TODO GK: don't assume hemispheres
    RR = g.subgraph(RRnodes)
    RRdegs = [RR.degree(**wt_args)[n] for n in RRnodes]

    LRnodes = g.nodes()
    ipso_list = LLdegs + RRdegs
    degs = [g.degree(**wt_args)[n] for n in LRnodes]
    contra_deg[subj] = [a_i - b_i for a_i, b_i in zip(degs, ipso_list)]
    ipso_deg[subj] = ipso_list
# import pdb; pdb.set_trace()

In [20]:
deg = {'total_deg': total_deg,
       'ipso_deg': ipso_deg,
       'contra_deg': contra_deg}
print("Computing: Degree Sequence")
# write(outdir, 'degree_distribution', deg, atlas)
statsDict['degree_distribution'] = deg
show_means(total_deg)

Computing: Degree Sequence
Subject Means: 26.47, 21.71


#  Edge Weights

In [21]:
if modality == 'dwi':
    print("Computing: Edge Weight Sequence")
    temp_ew = OrderedDict((s, [graphs[s].get_edge_data(e[0], e[1])['weight'] 
                               for e in graphs[s].edges()]) for s in graphs)
    ew = temp_ew
    # write(outdir, 'edge_weight', ew, atlas)
    statsDict['edge_weight'] = ew
    show_means(temp_ew)
else:
    temp_pl = OrderedDict()
    print("Computing: Path Length Sequence")
    nxappl = nx.all_pairs_dijkstra_path_length
    for s in graphs:
        apd = nxappl(graphs[s])
        # iterate over the nodes to find the average distance to each node
        avg_path = [np.nanmean(v.values()) for k, v in apd.iteritems()]
        temp_pl[s] = np.array(avg_path)
    pl = temp_pl
    # write(outdir, 'path_length', pl, atlas)
    statsDict['path_length'] = pl
    show_means(pl)

Computing: Edge Weight Sequence
Subject Means: 1042.34, 1092.18


# Eigen Values

In [22]:
print("Computing: Eigen Value Sequence")
laplac = OrderedDict((subj, nx.normalized_laplacian_matrix(graphs[subj])) 
                     for subj in graphs)
eigs = OrderedDict((subj, np.sort(np.linalg.eigvals(laplac[subj].A))[::-1]) 
                   for subj in graphs)

Computing: Eigen Value Sequence


In [23]:
# write(outdir, 'eigen_sequence', eigs, atlas)
statsDict['eigen_sequence'] = eigs

In [24]:
print("Subject Maxes: " + ", ".join(["%.2f" % np.max(eigs[key]) 
                                     for key in eigs.keys()]))

Subject Maxes: 1.43, 1.45


# Betweenness Centrality

In [25]:
print("Computing: Betweenness Centrality Sequence")
nxbc = nx.algorithms.betweenness_centrality  # For PEP8 line length...
temp_bc = OrderedDict((subj, nxbc(graphs[subj], **wt_args).values())
                      for subj in graphs)
centrality = temp_bc
# write(outdir, 'betweenness_centrality', centrality, atlas)
statsDict['betweenness_centrality'] = centrality
show_means(temp_bc)

Computing: Betweenness Centrality Sequence
Subject Means: 0.01, 0.01


# Mean connectome

In [26]:
print("Computing: Mean Connectome")
nxnp = nx.to_numpy_matrix
adj = OrderedDict((subj, nxnp(graph, nodelist=sorted(graph.nodes())))
                  for subj, graph in graphs.iteritems())
mat = np.zeros(adj.values()[0].shape)
for subj in adj:
    mat += adj[subj]
mat = mat/len(adj.keys())
# write(outdir, 'study_mean_connectome', mat, atlas)
statsDict['study_mean_connectome'] = mat

Computing: Mean Connectome


In [27]:
list(statsDict.keys())

['degree_distribution',
 'betweenness_centrality',
 'locality_statistic',
 'eigen_sequence',
 'number_non_zeros',
 'clustering_coefficients',
 'edge_weight',
 'study_mean_connectome']

## Confirm if above code produces the same output as compute_metrics() in ndmg

In [28]:
qg.compute_metrics(fs, outdir, atlas = "atlas")

Computing: NNZ
Sample Mean: 1337.00
Computing: Degree Sequence
Subject Means: 26.47, 21.71
Computing: Edge Weight Sequence
Subject Means: 1042.34, 1092.18
Computing: Clustering Coefficient Sequence
Subject Means: 0.65, 0.65
Computing: Max Local Statistic Sequence
Subject Means: 398574.53, 249819.52
Computing: Eigen Value Sequence
Subject Maxes: 1.43, 1.45
Computing: Betweenness Centrality Sequence
Subject Means: 0.01, 0.01
Computing: Mean Connectome


# Plotting

In [29]:
dictKeys = list(statsDict.keys())
dictKeys = sorted(dictKeys, key = str)
dictKeys

['betweenness_centrality',
 'clustering_coefficients',
 'degree_distribution',
 'edge_weight',
 'eigen_sequence',
 'locality_statistic',
 'number_non_zeros',
 'study_mean_connectome']

In [30]:
if modality == 'dwi':
    modtit = "DWI"
    order = [0, 1, 2, 5, 4, 3, 6, 7]
    dictKeys = [dictKeys[o] for o in order]
    labs = ['Betweenness Centrality', 'Clustering Coefficient', 'Degree',
            'Locality Statistic-1', 'Eigenvalue', 'Edge Weight',
            'Number of Non-zeros', 'Mean Connectome']
else:
    modtit = "fMRI"
    order = [0, 1, 2, 6, 4, 3, 5, 7]
    dictKeys = [dictKeys[o] for o in order]
    labs = ['Betweenness Centrality', 'Clustering Coefficient', 'Degree',
            'Average Path Length', 'Locality Statistic-1', 'Eigenvalue',
            'Number of Non-zeros', 'Mean Connectome']

In [31]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [32]:
def make_panel_plot(statsDict, outf, dataset=None, atlas=None, minimal=True, 
                    log=True, hemispheres=True, modality='dwi'):
    
    dictKeys = list(statsDict.keys())
    dictKeys = sorted(dictKeys, key = str)
    
    if modality == 'dwi':
        modtit = "DWI"
        order = [0, 1, 2, 5, 4, 3, 6, 7]
        dictKeys = [dictKeys[o] for o in order]
        labs = ['Betweenness Centrality', 'Clustering Coefficient', 'Degree',
                'Locality Statistic-1', 'Eigenvalue', 'Edge Weight',
                'Number of Non-zeros', 'Mean Connectome']
        
    else:
        modtit = "fMRI"
        order = [0, 1, 2, 6, 4, 3, 5, 7]
        dictKeys = [dictKeys[o] for o in order]
        labs = ['Betweenness Centrality', 'Clustering Coefficient', 'Degree',
                'Average Path Length', 'Locality Statistic-1', 'Eigenvalue',
                'Number of Non-zeros', 'Mean Connectome']
    traces = list(())
    
    for idx, curr in enumerate(dictKeys):
        dat = statsDict[dictKeys[idx]]
        if dictKeys[idx] == 'number_non_zeros':
            fig = plot_rugdensity(dat.values())
        elif dictKeys[idx] == 'edge_weight':
            edges = np.max([len(dat[i]) for i in dat.keys()])
            fig = plot_series(dat.values(), sort=True)
        elif dictKeys[idx] == 'degree_distribution':
            fig = plot_degrees(dat, hemi=hemispheres)
            if hemispheres:
                maxdat = np.max([np.max(dat[key][k]) 
                                 for key in dat.keys()
                                 for k in dat[key]])
                anno = [dict(x=dims/3,
                             y=4*float(maxdat/7),
                             xref='x3',
                             yref='y3',
                             text='ipsilateral',
                             showarrow=False,
                             font=dict(color='rgba(0.0,0.0,0.0,0.6)',
                                       size=14)),
                        dict(x=dims/3,
                             y=3.7*float(maxdat/7),
                             xref='x3',
                             yref='y3',
                             text='contralateral',
                             showarrow=False,
                             font=dict(color='rgba(0.11,0.62,0.47,0.6)',
                                       size=14))]
        elif dictKeys[idx] == 'study_mean_connectome':
            if log:
                dat = np.log10(dat+1)
            fig = plot_heatmap(dat, name=labs[idx])
        else:
            dims = len(dat.values()[0])
            fig = plot_series(dat.values())
        traces += [fig_to_trace(fig)]

    multi = traces_to_panels(traces)
    for idx, curr, in enumerate(dictKeys):
        key = 'axis%d' % (idx+1)
        d = multi.layout['x'+key]['domain']
        multi.layout['x'+key]['domain'] = [d[0], d[1]-0.0125]
        multi.layout['x'+key]['zeroline'] = False
        multi.layout['y'+key]['zeroline'] = False
        multi.layout['y'+key]['title'] = ''
        multi.layout['x'+key]['title'] = 'Node'
        multi.layout['x'+key]['nticks'] = 3
        multi.layout['y'+key]['nticks'] = 3
        if (idx in [0, 1, 2, 3, 4, 5] and modality == 'func') or (idx in [0, 1, 2, 3, 4, 5] and modality == 'dwi'):
            multi.layout['x'+key]['range'] = [1, dims]
            multi.layout['x'+key]['tickvals'] = [1, dims/2, dims]
            if idx in [2]:
                if hemispheres:
                    multi.layout['annotations'] = anno
            elif log:
                multi.layout['y'+key]['type'] = 'log'
                multi.layout['y'+key]['title'] += 'log'
        if idx in [5] and modality == 'dwi':
            multi.layout['x'+key]['range'] = [1, edges]
            multi.layout['x'+key]['tickvals'] = [1, edges/2, edges]
            multi.layout['x'+key]['title'] = 'Edge'
        if (idx in [4] and modality == 'dwi') or (idx in [5] and modality == 'func'):
            multi.layout['x'+key]['range'] = [1, dims]
            multi.layout['x'+key]['tickvals'] = [1, dims/2, dims]
            multi.layout['x'+key]['title'] = 'Dimension'
        multi.layout['y'+key]['title'] += labs[idx]
        if (idx in [6]):
            multi.layout['y'+key]['title'] = 'Relative Probability'
            multi.layout['x'+key]['title'] = labs[idx]
        if idx in [7]:
            multi.layout['y'+key]['title'] = None
            multi.layout['x'+key]['title'] = labs[idx]
            multi.layout['y'+key]['autorange'] = 'reversed'
            multi.layout['x'+key]['tickvals'] = [0, dims/2-1, dims-1]
            multi.layout['y'+key]['tickvals'] = [0, dims/2-1, dims-1]
            multi.layout['x'+key]['ticktext'] = [1, dims/2, dims]
            multi.layout['y'+key]['ticktext'] = [1, dims/2, dims]
            if log:
                multi.layout['x'+key]['title'] += ' (log10)'
    if dataset is not None and atlas is not None:
        if atlas == 'desikan':
            atlas = atlas.capitalize()
        tit = "{} Dataset ({} parcellation), {} Group Analysis".format(dataset,
                atlas, modtit)
    else:
        tit = "{} Group Analysis".format(modtit)
    multi.layout['title'] = tit
    # iplot(multi, validate=False)

    if minimal:
        locs = [idx for idx, d in enumerate(multi.data) if d['yaxis'] == 'y8']
        for l in locs:
            multi.data[l] = {}
        multi.layout['x'+key]['title'] = ''
        multi.layout['y'+key]['title'] = ''
        multi = panel_invisible(multi, 8)

    plot(multi, validate=False, filename=outf+'.html')

In [33]:
make_panel_plot(statsDict, outf = "plot")

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]  [ (1,4) x4,y4 ]
[ (2,1) x5,y5 ]  [ (2,2) x6,y6 ]  [ (2,3) x7,y7 ]  [ (2,4) x8,y8 ]



## Make sure the output is the same as ndmg

In [34]:
from ndmg.stats import qa_graphs_plotting as qgp

In [35]:
qgp.make_panel_plot(basepath = outdir, outf = "plot")

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]  [ (1,4) x4,y4 ]
[ (2,1) x5,y5 ]  [ (2,2) x6,y6 ]  [ (2,3) x7,y7 ]  [ (2,4) x8,y8 ]

